In [13]:
def load_FRCNNmodel_cpu(cfg, model_path): 
    print("load FRCNN model")
    Trainer =DefaultTrainer
    model = Trainer.build_model(cfg)    
    DetectionCheckpointer(model).resume_or_load(model_path, resume=False)
    return model.model.cpu()
def load_TSmodel_cpu(cfg, model_path):
    Trainer =PTrainer
    model = Trainer.build_model(cfg)
    model_teacher = Trainer.build_model(cfg)
    ensem_ts_model = EnsembleTSModel(model_teacher, model)    
    DetectionCheckpointer(ensem_ts_model).resume_or_load(model_path, resume=False)
    return ensem_ts_model.modelStudent.cpu()
def load_SOmodel( cfg, model_path):
    print("load source-only pt model")
    Trainer= PTrainer_sourceonly   
    if cfg.FEDSET.DYNAMIC: 
        fedma_model = torch.load(model_path)
        backbone_dim = FedUtils.get_backbone_shape(fedma_model)

        cfg.defrost()
        #cfg.MODEL.WEIGHTS = model_path                    
        cfg.BACKBONE_DIM = backbone_dim        
        cfg.freeze

        model = Trainer.build_model(cfg,cfg.BACKBONE_DIM,False)
    else:
        model = Trainer.build_model(cfg) 
    DetectionCheckpointer(model).resume_or_load(model_path, resume=False)
    return model
    

def get_trainer( trainer_name, cfg, model_path):
    if trainer_name == "pt":
        return load_TSmodel(cfg, model_path)  
    elif trainer_name == "sourceonly":
        return load_SOmodel(cfg, model_path)              
    elif trainer_name == "default":
        return load_FRCNNmodel(cfg, model_path)       
    else:
        raise ValueError("Trainer Name is not found.")

In [3]:
from Nb_utils import *
cfg_path = "configs/multi-teacher/skf2c_foggy_sourceonly_FedMAbackbone.yaml"

cfg = setup_all(cfg_path)
model_path_list = cfg.MODEL.TEACHER_PATH  
teacher_trainer = cfg.MODEL.TEACHER_TRAINER

In [27]:
model_path_list

['output/FedMA_skf2c_source-only/FedMA_VOC2007_sim1_12.pth',
 'output/FedMA_skf2c_source-only/FedMA_VOC2007_kitti1_12.pth',
 'output/FedMA_skf2c_source-only/FedMA_VOC2007_foggytrain1_12.pth']

In [26]:
torch.load(model_path_list[0])

OrderedDict([('backbone.vgg_block1.0.conv1.weight',
              tensor([[[[ 4.2947e-01,  3.7347e-01, -6.1360e-02],
                        [ 2.7477e-01,  3.8681e-02, -3.6722e-01],
                        [-5.7468e-02, -2.6225e-01, -3.5010e-01]],
              
                       [[ 5.5038e-01,  4.4007e-01, -8.1387e-02],
                        [ 3.4574e-01,  4.0632e-02, -4.5350e-01],
                        [-5.8635e-02, -3.3067e-01, -4.8503e-01]],
              
                       [[ 4.8002e-01,  4.0855e-01, -6.5146e-02],
                        [ 3.1048e-01,  5.0202e-02, -4.0338e-01],
                        [-5.0872e-02, -2.8523e-01, -4.1852e-01]]],
              
              
                      [[[ 1.1727e-01,  1.6206e-01,  1.3569e-01],
                        [ 1.4835e-01,  2.0230e-01,  1.6169e-01],
                        [ 1.2934e-01,  1.7157e-01,  1.3871e-01]],
              
                       [[ 2.0877e-02,  4.7341e-02,  4.1854e-02],
                       

In [28]:
torch.load('output/FedMA_skf2c_source-only/VOC2007_foggytrain1_12/model_final.pth')

{'model': OrderedDict([('backbone.vgg_block1.0.conv1.weight',
               tensor([[[[ 4.2947e-01,  3.7347e-01, -6.1360e-02],
                         [ 2.7477e-01,  3.8681e-02, -3.6722e-01],
                         [-5.7468e-02, -2.6225e-01, -3.5010e-01]],
               
                        [[ 5.5038e-01,  4.4007e-01, -8.1387e-02],
                         [ 3.4574e-01,  4.0632e-02, -4.5350e-01],
                         [-5.8635e-02, -3.3067e-01, -4.8503e-01]],
               
                        [[ 4.8002e-01,  4.0855e-01, -6.5146e-02],
                         [ 3.1048e-01,  5.0202e-02, -4.0338e-01],
                         [-5.0872e-02, -2.8523e-01, -4.1852e-01]]],
               
               
                       [[[ 1.1727e-01,  1.6206e-01,  1.3569e-01],
                         [ 1.4835e-01,  2.0230e-01,  1.6169e-01],
                         [ 1.2934e-01,  1.7157e-01,  1.3871e-01]],
               
                        [[ 2.0877e-02,  4.7341e-02,  4.1854e-

In [16]:
from FLpkg import FedUtils 
model_list=[ get_trainer(teacher_trainer, cfg, model_teacher_path) for model_teacher_path in model_path_list]

load source-only pt model
load source-only pt model
load source-only pt model


In [18]:
model_list[1]

GuassianGeneralizedRCNN(
  (backbone): VGG(
    (vgg_block1): Sequential(
      (0): VGGBlock(
        (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (vgg_block2): Sequential(
      (0): VGGBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (vgg_block3): Sequential(
      (0): VGGBlock(
        (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(256, 262, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(262, 262, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [19]:
cfg_path = "configs/202405_multiclass/fin04_ma02.yaml"

cfg = setup_all(cfg_path)
model_path_list = cfg.MODEL.TEACHER_PATH  
teacher_trainer = cfg.MODEL.TEACHER_TRAINER

In [23]:
model_path_list

['output/ma02_skf2c_cla8_20240411/VOC2007_sim1_12/model_final.pth',
 'output/ma02_skf2c_cla8_20240411/VOC2007_kitti5_12/model_final.pth',
 'output/ma02_skf2c_cla8_20240411/VOC2007_foggytrain_12/model_final.pth']

In [24]:
torch.load(model_path_list[0])

{'model': OrderedDict([('backbone.vgg_block1.0.conv1.weight',
               tensor([[[[ 4.2947e-01,  3.7347e-01, -6.1360e-02],
                         [ 2.7477e-01,  3.8681e-02, -3.6722e-01],
                         [-5.7468e-02, -2.6225e-01, -3.5010e-01]],
               
                        [[ 5.5038e-01,  4.4007e-01, -8.1387e-02],
                         [ 3.4574e-01,  4.0632e-02, -4.5350e-01],
                         [-5.8635e-02, -3.3067e-01, -4.8503e-01]],
               
                        [[ 4.8002e-01,  4.0855e-01, -6.5146e-02],
                         [ 3.1048e-01,  5.0202e-02, -4.0338e-01],
                         [-5.0872e-02, -2.8523e-01, -4.1852e-01]]],
               
               
                       [[[ 1.1727e-01,  1.6206e-01,  1.3569e-01],
                         [ 1.4835e-01,  2.0230e-01,  1.6169e-01],
                         [ 1.2934e-01,  1.7157e-01,  1.3871e-01]],
               
                        [[ 2.0877e-02,  4.7341e-02,  4.1854e-

In [20]:
model_list_wy=[ get_trainer(teacher_trainer, cfg, model_teacher_path) for model_teacher_path in model_path_list]

load source-only pt model


AttributeError: 'dict' object has no attribute 'shape'